In [ ]:
import datetime
import h5py
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path
import tables
from tqdm import tqdm
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
all_data= pd.read_csv(r"C:\Users\ag-bahl\Desktop\\all_dataframes_stimulus_09_11.csv")
print('done all data')

In [ ]:
all_data


In [ ]:
# reorder the data according to first fish_name, then trial.
all_data.sort_values(by=['fish_name', 'trial'], inplace=True, ascending=True)
all_data

In [ ]:
# split the data into optimum gray and stimulus.

In [ ]:
# then get a dataframe with streak lengths, for:
# 1. All data
# 2. Optimum gray only
# 3. Plaids, or whatever stimulus.


In [ ]:
# then get the values, from Ashrit's functions as you got berfore, mean and sem through a trial

In [ ]:
# Then we get similar to Ashrit's function, every other variable that I consider important

In [ ]:
# Then histograms, loads hahaha - the turning angle distributions for each configuration

In [ ]:
# then measure the streak length, and first a histogram, then the CDF and PDF whatever.
# and then for each combination of stimuli, you add this after you get 8 direction data...